In [108]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [109]:
# Filepath of the dataset
pre_filepath = "../../../../../../Volumes/Seagate Backup Plus Drive/Documents/Kaggle Datasets/"

In [135]:
# Paramaters definitions
VALIDATION_SPLIT = 35000
BATCH_SIZE = 1
EPOCHS = 5
n_batches = VALIDATION_SPLIT//BATCH_SIZE
n_classes = int(y_train.shape[1])

In [148]:
X_train_placeholder = tf.placeholder(tf.float32, [BATCH_SIZE, 64, 64, 3])
y_train_placeholder = tf.placeholder(tf.uint8, [BATCH_SIZE, 17])
X_train_batch = tf.train.batch([X_train], batch_size=BATCH_SIZE, capacity=BATCH_SIZE)
y_train_batch = tf.train.batch([y_train], batch_size=BATCH_SIZE, capacity=BATCH_SIZE)

In [137]:
# Helper wrappers
def conv2d(x, W, b, strides=1):
    return_val = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    return_val = tf.nn.bias_add(return_val, b)
    return tf.nn.relu(return_val)

def maxpool(x, pool_size=2):
    return tf.nn.max_pool(x, [1,pool_size,pool_size,1], [1,pool_size,pool_size,1], padding='SAME')

def fc(x, W, b):
    return_val = tf.matmul(x, W)
    return_val = tf.nn.bias_add(return_val, b)
    return tf.nn.relu(return_val)

In [138]:
# Build the Tensorflow model!
def model(images, weights, biases, dropout=0.8):
    """
    Defines the image classification model
    
    Inputs:
        images: entire training set of images
        input_shape: dimensions of input as a tuple
    
    Outputs logits
    """
    
    # Apply convolution and pooling to each layer
    conv1 = conv2d(images, weights['conv1'], biases['conv1'])  
    conv1 = maxpool(conv1)
    
    conv2 = conv2d(conv1, weights['conv2'], biases['conv2'])
    conv2 = maxpool(conv2)
    
    conv3 = conv2d(conv2, weights['conv3'], biases['conv3'])
    conv3 = maxpool(conv3)
    
    # Apply dropout
    conv3 = tf.nn.dropout(conv3, dropout)
    
    # First reshape output of conv3 into a vector
    conv3_vec = tf.reshape(conv3, [1, 8*8*128])
    
    # FC layers
    fc1 = fc(conv3_vec, weights['fc1'], biases['fc1'])
    # Then apply dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    fc2 = fc(fc1, weights['fc2'], biases['fc2'])
    
    # Return logits which is a vector of 17 class scores
    return fc2

In [139]:
weights = {'conv1':tf.Variable(tf.random_normal([3,3,3,32])), # 3 by 3 convolution, 3 channels (depth), 32 outputs
           'conv2':tf.Variable(tf.random_normal([3,3,32,64])), # 3 by 3 convolution, 32 inputs, 64 outputs
           'conv3':tf.Variable(tf.random_normal([3,3,64,128])), # 3 by 3 convolution, 64 inputs, 128 outputs
           'fc1':tf.Variable(tf.random_normal([8*8*128,1024])), 
           'fc2':tf.Variable(tf.random_normal([1024,n_classes]))}

biases = {'conv1':tf.Variable(tf.random_normal([32])),
          'conv2':tf.Variable(tf.random_normal([64])),
          'conv3':tf.Variable(tf.random_normal([128])),
          'fc1':tf.Variable(tf.random_normal([1024])),
          'fc2':tf.Variable(tf.random_normal([n_classes]))}

In [140]:
# Instantiate the model
pred_logits = model(X_train, weights, biases)

In [141]:
# Loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,
                                                              logits=pred_logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [142]:
# Evaluate the model

# Get the prediction from the highest valued logit
prediction = tf.argmax(pred_logits)
prediction = tf.cast(prediction, tf.uint8)

# Compute array of bools that indicate whether the prediction was correct
pred_correct = tf.equal(y_train, prediction)

# Compute accuracy
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

In [143]:
# Load X_train and y_train arrays from disk
# X_train = np.load(pre_filepath + "/Planet/model-data/X_train_arr.npy")
# y_train = np.load(pre_filepath + "/Planet/model-data/y_train_arr.npy")

# Load X_train and y_train arrays from disk
X_train_arr = np.load("model-data/X_train_arr.npy")
y_train_arr = np.load("model-data/y_train_arr.npy")

In [144]:
# Splitting up the training data into training and validation sets
X_train_arr = X_train_arr[:VALIDATION_SPLIT]
y_train_arr = y_train_arr[:VALIDATION_SPLIT]

X_valid_arr = X_train_arr[VALIDATION_SPLIT:]
y_valid_arr = y_train_arr[VALIDATION_SPLIT:]

In [149]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCHS):
        training_loss = 0
        train_accuracies = np.array([])
        for image in range(VALIDATION_SPLIT):
            # Extract the next BATCH_SIZE images and labels
            X_train_img = X_train_arr[image]
            y_train_img = y_train_arr[image]
            
            # Run the model
            feed_dict_train = {X_train:X_train_img, y_train:y_train_img}
            loss, accuracy = sess.run([loss, accuracy], feed_dict=feed_dict_train)
            np.append(train_accuracies, accuracy)
            
            training_loss += loss
        
        # Compute training accuracy
        train_accuracy = np.mean(train_accuracies)
        
        # Compute validation accuracy
        feed_dict_valid = {X_train_placeholder:X_valid_arr, y_train_placeholder:y_valid_arr}
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
        print("Epoch: " + epoch)
        print("Training loss: " + training_loss + "Training accuracy: " + train_accuracy)
        print("Validation loss: " + valid_loss + "Validation accuracy: " + valid_accuracy)

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
X_train_arr.shape

In [103]:
X_train_arr[1:2].shape

(1, 64, 64, 3)